In [ ]:
HTML(read(open("style.html"), String))

These are the new implementations, remove old ones later

In [1]:
using NBInclude
@nbinclude("Piece-Weight and Simple Evaluation.ipynb")

evaluate2 (generic function with 1 method)

In [21]:
function valueOf(piece, square)
    if piece === EMPTY
        return 0
    end
    matrix = get_matrix_from_piece(piece, piece_dict)
    weight = get_weight_from_piece(piece, piece_weight)
    piece_value = matrix[rank(square).val][file(square).val] + weight
    return piece_value
end

# assert target_move to be legal
function evaluate_incremental(source_score_white, source_score_black, source_board, target_move)
    # possible incremental changes
    # - move 1 piece (just move)
    # - move 1 piece and remove 1 piece of the opposite color on the target location (capture)
    # - move 1 piece and remove 1 piece of the opposite color on the target location +1 rank in center direction (en passent)
    # - move 1 piece and replace it (promotion)
    # - move two pieces (casteling)
    
    # dispatch action
    #   if: ischeckmate
    #     return +- infinity
    #
    #   if: isdraw
    #     return 0
    #
    #   update score by - captured piece and new piece position
    #
    #   if: move is promition
    #     update score by removing pawn & adding promition piece at target location
    #
    #   if: piece is king & move is castle
    #     (casteling can only be initiated by moving the king)
    #     update score by new king & rook positions
    #
    #   if: piece is pawn & move is en passent
    #     update score by - captured piece and new pawn position 
    #       
    
    color_to_move = sidetomove(source_board)
    
    source_square = from(target_move)
    piece_to_move = pieceon(source_board, source_square)
    destination_square = to(target_move)
    piece_to_capture = pieceon(source_board, destination_square)
    
    undoinfo = domove!(source_board, target_move)
    
    if ischeckmate(source_board)
        undomove!(source_board, undoinfo)
        if color_to_move == WHITE
            return 200000, -200000
        else
            return -200000, 200000
        end
    end
    
    if isdraw(source_board)
        undomove!(source_board, undoinfo)
        return 0, 0
    end
    
    undomove!(source_board, undoinfo)
    
    # for later use
    destination_square_piece_value = valueOf(piece_to_move, destination_square)
    
    # remove & readd piece, remove captured piece (may be nothing)
    if color_to_move == WHITE
        source_score_white -= valueOf(piece_to_move, source_square)
        source_score_white += destination_square_piece_value
        source_score_black -= valueOf(piece_to_capture, destination_square)
    else
        source_score_black -= valueOf(piece_to_move, source_square)
        source_score_black += destination_square_piece_value
        source_score_white -= valueOf(piece_to_capture, destination_square)
    end
    
    piece_to_move_type = ptype(piece_to_move)
    
    # casteling? -> also count for rook
    # casteling is the only case where distance == 2
    if piece_to_move_type === KING
        if distance(source_square, destination_square) == 2
            destination_square_file = file(destination_square)

            # C & G are the only files where a castling king can move to
            if destination_square_file == FILE_C
                # rook moved fom FILE_A to FILE_D

                # white queen side castle
                if color_to_move == WHITE 
                    source_score_white -= valueOf(PIECE_WR, SQ_A1)
                    source_score_white += valueOf(PIECE_WR, SQ_D1)
                # black king side castle
                else
                    source_score_black -= valueOf(PIECE_BR, SQ_A8)
                    source_score_black += valueOf(PIECE_BR, SQ_D8)
                end
            elseif destination_square_file == FILE_G 
                # rook moved fom FILE_H to FILE_F

                # white king side castle
                if color_to_move == WHITE 
                    source_score_white -= valueOf(PIECE_WR, SQ_H1)
                    source_score_white += valueOf(PIECE_WR, SQ_F1)
                # black queen side castle
                else
                    source_score_black -= valueOf(PIECE_BR, SQ_H8)
                    source_score_black += valueOf(PIECE_BR, SQ_F8)
                end
            end
        end
            
        # break early
        return source_score_white, source_score_black
    end
    
    # casteling? -> also count for replaced piece
    if ispromotion(target_move)
        promoted_piece = promotion(move)
        promoted_piece_value = valueOf(Piece(color_to_move, promoted_piece), piece_dict, piece_weight)
        # replace white piece with promoted piece
        if color_to_move === WHITE
            source_score_white -= destination_square_piece_value
            source_score_white += promoted_piece_value
        # replace black piece with promoted piece
        else
            source_score_black -= destination_square_piece_value
            source_score_black += promoted_piece_value
        end
            
        # break early
        return source_score_white, source_score_black
    end
    
    # is pawn & source file != dest file & piece_to_capture == nothing -> en passent
    # if piece_to_move_type === PAWN && file(source_square) != file(destination_square) && piece_to_capture === nothing
    en_passent_capture_square = epsquare(source_board)
    if en_passent_capture_square != SQ_NONE
        # remove black pawn
        if color_to_move === WHITE
            #source_score_black -= valueOf(PIECE_BP, Square(file(destination_square), rank(source_square)))
            source_score_black -= valueOf(PIECE_BP, en_passent_capture_square)
        # remove white pawn
        else
            #source_score_white -= valueOf(PIECE_WP, Square(file(destination_square), rank(source_square)))
            source_score_white -= valueOf(PIECE_WP, en_passent_capture_square)
        end
    end
    
    return source_score_white, source_score_black
end

evaluate_incremental (generic function with 1 method)

In [ ]:
b = startboard()

In [ ]:
b = startboard()
domove!(b, "e2e4")
println(evaluate_for_color(b, WHITE, piece_dict, piece_weight))
println(evaluate_for_color(b, BLACK, piece_dict, piece_weight))
domove!(b, "d7d5")
println(evaluate_for_color(b, WHITE, piece_dict, piece_weight))
println(evaluate_for_color(b, BLACK, piece_dict, piece_weight))
domove!(b, "e4d5")
println(evaluate_for_color(b, WHITE, piece_dict, piece_weight))
println(evaluate_for_color(b, BLACK, piece_dict, piece_weight))

In [ ]:
b = startboard()
score_white = evaluate_for_color(b, WHITE, piece_dict, piece_weight)
score_black = evaluate_for_color(b, BLACK, piece_dict, piece_weight)
score_white, score_black = evaluate_incremental(score_white, score_black, b, movefromstring("e2e4"))
println(score_white)
println(score_black)
domove!(b, "e2e4")
score_white, score_black = evaluate_incremental(score_white, score_black, b, movefromstring("d7d5"))
println(score_white)
println(score_black)
domove!(b, "d7d5")
score_white, score_black = evaluate_incremental(score_white, score_black, b, movefromstring("e4d5"))
println(score_white)
println(score_black)

In [3]:
function minimax_incremental(score_white, score_black, board, requiredDepth)

    best_move = nothing
    
    function minimax(score_white, score_black, board, depth)
        possibleMoves = moves(board)
        if depth == 0 || size(possibleMoves) == 0
            if sidetomove(board) == WHITE
                return score_white - score_black
            else
                return score_black - score_white
            end
        end
        maxValue = 1 << 63;
        for move in possibleMoves
            new_score_white, new_score_black = evaluate_incremental(score_white, score_black, board, move)
            undoInfo = domove!(board, move)
            value = -minimax(new_score_white, new_score_black, board, depth - 1)
            undomove!(board, undoInfo)
            if value > maxValue
                maxValue = value
                if depth == requiredDepth
                    println("Best move: ", move, " score ", value)
                    best_move = move
                end
            end
        end
        recycle!(possibleMoves)
        return maxValue
    end
    
    minimax(score_white, score_black, board, requiredDepth)
    return best_move
end

minimax_incremental (generic function with 1 method)

In [ ]:
struct MiniMaxIncAI 
    depth
end

function next_move(board, ai::MiniMaxIncAI)
    score_white = evaluate_for_color(board, WHITE, piece_dict, piece_weight)
    score_black = evaluate_for_color(board, WHITE, piece_dict, piece_weight)
    return minimax_incremental(score_white, score_black, board, ai.depth), ""
end

In [10]:
using NBInclude
@nbinclude("BasicGame.ipynb")

In [ ]:
play_game(MiniMaxIncAI(3))

In [17]:
struct MiniMaxProgressiveIncAI 
    maxTime
    maxDepth
end

function next_move(board, ai::MiniMaxProgressiveIncAI)
    score_white = evaluate_for_color(board, WHITE, piece_dict, piece_weight)
    score_black = evaluate_for_color(board, WHITE, piece_dict, piece_weight)
    
    function eval_minimax_incremental(with_depth)
         println("New search depth: ", with_depth)
         return minimax_incremental(score_white, score_black, board, with_depth)
    end
    
    deepest_finished_depth = 0
    best_move = nothing
    remaining_time = ai.maxTime
    last_time = 0
    
    while last_time * 2 < remaining_time
        last_time = @elapsed begin 
            best_move = eval_minimax_incremental(deepest_finished_depth + 1)
        end
        deepest_finished_depth += 1
        remaining_time -= last_time
    end
    
    if best_move !== nothing
        undoinfo = domove!(board, best_move)
        score_white = evaluate_for_color(board, WHITE, piece_dict, piece_weight)
        score_black = evaluate_for_color(board, BLACK, piece_dict, piece_weight)
        undomove!(board, undoinfo)
    end
    
    return best_move, "Depth: " * string(deepest_finished_depth) * "\nWhite: " * string(score_white) * "\nBlack: " * string(score_black)
end

next_move (generic function with 3 methods)

In [22]:
play_game(MiniMaxProgressiveIncAI(4, 5))

Board (r1bqkbnr/pppppppp/2n5/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  -  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  n  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R